# Notebook 1 : Transformer

Le but de ce notebook est d'importer les données des datasets iSarcasmEval et de tester les transformer (tel que donné par *“Attention Is All You Need”*) afin de mesurer l'accuracy de base du projet et quantifier les futures améliorations

### Etape 1 : Importer les librairies utiles au projet

In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd

Test de Pytorch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


### Etape 2 : Chargeons les données

In [11]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import optim


class iSarcasmDataset(Dataset):
  def __init__(self, sentence, labels):
        self.sentence = sentence
        self.labels = labels

  def __len__(self):
        return len(self.sentence)

  def __getitem__(self, index):
        x = self.sentence[index]
        y = self.labels[index]

        return x, y
    
    

print("--- Chargement des datasets ---")


#On donne les informations du dataset
directory = "dataset/"
train_name = "train.En.csv"
test_name = "task_A_En_test.csv"


#On défini les paramètres du dataloader
params = {'batch_size': 100,
          'shuffle': False,
          'num_workers': 1}


#On créé nos données de test
panda_Train = pd.read_csv(directory+train_name, index_col=[0])
X_train = panda_Train['tweet']
y_train = panda_Train['sarcastic']


#On créé notre dataset pyTorch
train_iSarcasm = iSarcasmDataset(X_train, y_train)
training_generator = DataLoader(train_iSarcasm, **params)


#On regarde que tout soit bien chargé
for i, (batch, labels) in enumerate(training_generator):
    if i > 0:
        break
    print("Batch size : "+str(len(batch)))
    
#On va créer nos données de validation
panda_Test = pd.read_csv(directory+test_name, index_col=[0])
X_test = panda_Test.index
y_test = panda_Test['sarcastic']

#On créé notre dataset pyTorch
test_iSarcasm = iSarcasmDataset(X_test, y_test)
test_generator = DataLoader(test_iSarcasm, **params)

#On regarde que tout soit bien chargé
for i, (batch, labels) in enumerate(test_generator):
    if i > 0:
        break
    print("Batch size : "+str(len(batch)))
    
print("--- Chargement des datasets terminé ---")

--- Chargement des datasets ---
Batch size : 100
Batch size : 100
--- Chargement des datasets terminé ---


On va maintenant réaliser un transformer de base sur nos données. Le réseau de neuronne aura la structure suivante : couche Embeddings, couche transformer, couche Dense pour la prédiction du prochain mot.

In [ ]:
# On créé les Embeddings avec la méthode du Word2Vec

print("--- Création de la layer Embedding ---")

taille_embeddings = 10

list_of_words = []
for phrase in X_train:
    if not(pd.isna(phrase)):
        list_of_words.extend(phrase.split())
    
list_of_words = np.unique(list_of_words)
len_Vocabulary = len(list_of_words)

print("La longueur du vocabulaire est de : "+str(len_Vocabulary))

embedding = nn.Embedding(len_Vocabulary, taille_embeddings)

print("--- Création de la layer Embedding terminée ---")

On va maintenant réaliser un transformer classique pour voir l'accuracy témoin

In [ ]:
# On déclare notre modèle
transformer_model = nn.Transformer(nhead=16, num_encoder_layers=12)

# On se déclare une loss
loss_fn = nn.CrossEntropyLoss()

# On se déclare un optimiseur qui effectuera la descente de gradient
optimizer_pyt = optim.Adam(transformer_model.parameters())

# L'historique pour print plus tard
loss_history, train_accuracy_history, valid_accuracy_history = [], [], []

for i, (batch, labels) in enumerate(training_generator):

    logits = transformer_model(embedding(batch),labels)
    print(logits)
    preds = torch.argmax(logits, dim=-1)
    accuracy = torch.sum(preds == labels) / len(batch)

    loss = loss_fn(logits, labels)

    optimizer_pyt.zero_grad()
    loss.backward()
    optimizer_pyt.step()

    training_accuracy.append(accuracy)
    loss_history.append(loss.detach().numpy())

    if i % 20 == 0:
        print('batch {} / {} loss : {}'.format(i, len(training_generator), loss))